In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np

# Import the API key.
from config import g_key

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,bluff,NZ,-46.6000,168.3333,48.90,64,74,5.88,broken clouds
1,1,jamestown,US,42.0970,-79.2353,42.03,71,100,7.00,overcast clouds
2,2,victoria,HK,22.2855,114.1577,72.32,76,98,15.01,overcast clouds
3,3,port alfred,ZA,-33.5906,26.8910,62.19,79,100,11.52,overcast clouds
4,4,zhenlai,CN,45.8495,123.2973,47.41,19,100,7.02,overcast clouds


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,santa rosa,PH,14.3122,121.1114,84.18,75,36,7.63,scattered clouds
6,6,san cristobal,VE,7.7669,-72.2250,83.61,47,94,8.90,overcast clouds
10,10,belmonte,BR,-15.8631,-38.8828,75.63,92,100,11.45,moderate rain
18,18,butaritari,KI,3.0707,172.7902,79.86,80,46,12.03,light rain
22,22,mount isa,AU,-20.7333,139.5000,78.57,27,75,9.22,light rain
25,25,vaini,TO,-21.2000,-175.2000,77.16,88,40,5.75,scattered clouds
26,26,sapele,NG,5.8941,5.6767,87.55,60,87,7.76,overcast clouds
27,27,saint-philippe,RE,-21.3585,55.7679,78.39,83,60,13.04,light rain
36,36,coolidge,US,32.9778,-111.5176,77.67,9,0,5.75,clear sky
37,37,souillac,MU,-20.5167,57.5167,77.52,76,69,14.03,broken clouds


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
5,santa rosa,PH,scattered clouds,84.18,14.3122,121.1114,
6,san cristobal,VE,overcast clouds,83.61,7.7669,-72.2250,
10,belmonte,BR,moderate rain,75.63,-15.8631,-38.8828,
18,butaritari,KI,light rain,79.86,3.0707,172.7902,
22,mount isa,AU,light rain,78.57,-20.7333,139.5000,


In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    #print(params)
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df


,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
5,santa rosa,PH,scattered clouds,84.18,14.3122,121.1114,Erlyn’s Sta Rosa City Home
6,san cristobal,VE,overcast clouds,83.61,7.7669,-72.2250,Pirineos
10,belmonte,BR,moderate rain,75.63,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
18,butaritari,KI,light rain,79.86,3.0707,172.7902,Isles Sunset Lodge
22,mount isa,AU,light rain,78.57,-20.7333,139.5000,ibis Styles Mt Isa Verona
...,...,...,...,...,...,...,...
661,oranjestad,AW,clear sky,78.69,12.5240,-70.0270,Divi Village Golf & Beach Resort
665,port hedland,AU,clear sky,80.65,-20.3167,118.5667,The Esplanade Hotel
671,banjar,ID,overcast clouds,77.92,-8.1900,114.9675,The Hamsa Resort
672,baghdad,IQ,clear sky,76.78,33.3406,44.4009,Baghdad Hotel


In [22]:

hotel_df.isnull().sum()

# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
5,santa rosa,PH,scattered clouds,84.18,14.3122,121.1114,Erlyn’s Sta Rosa City Home
6,san cristobal,VE,overcast clouds,83.61,7.7669,-72.2250,Pirineos
10,belmonte,BR,moderate rain,75.63,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
18,butaritari,KI,light rain,79.86,3.0707,172.7902,Isles Sunset Lodge
22,mount isa,AU,light rain,78.57,-20.7333,139.5000,ibis Styles Mt Isa Verona
...,...,...,...,...,...,...,...
661,oranjestad,AW,clear sky,78.69,12.5240,-70.0270,Divi Village Golf & Beach Resort
665,port hedland,AU,clear sky,80.65,-20.3167,118.5667,The Esplanade Hotel
671,banjar,ID,overcast clouds,77.92,-8.1900,114.9675,The Hamsa Resort
672,baghdad,IQ,clear sky,76.78,33.3406,44.4009,Baghdad Hotel


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
5,14.3122,121.1114
6,7.7669,-72.2250
10,-15.8631,-38.8828
18,3.0707,172.7902
22,-20.7333,139.5000
...,...,...
661,12.5240,-70.0270
665,-20.3167,118.5667
671,-8.1900,114.9675
672,33.3406,44.4009


In [31]:
# 11a. Add a marker layer for each city to the map. 


max_temp = clean_hotel_df["Max Temp"]
cur_desc = clean_hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save map
plt.savefig("WeatherPy_vacation_map.png")

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

Figure(layout=FigureLayout(height='420px'))